# Case study - favourable words from restaurant reviews

In [69]:
# import the required libraries here
import numpy as np
import pandas as pd

## Most popular restaurant category
Hopefully you had a look at the data dictionary [here](https://www.yelp.com/dataset/documentation/main) and concluded that the business.json file was the best place to start to answer the question of what is the most popular category. We load the data in from file for you. If the data file is in your working directory, with this notebooks, this should work. Then we perform some typical quick looks at it.

In [70]:
# For convenience here, we are accessing the data in the working directory that contains our notebooks.
# Normal best practise is to keep your data separate, but this keeps things simple.
business = pd.read_json('yelp_academic_dataset_business.json', lines=True)

In [71]:
business.shape

(15585, 15)

In [72]:
business.head()

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
0,"{'Take-out': False, 'Wi-Fi': 'free', 'Good For...",O_X3PGhk3Y5JWVi866qlJg,"[Active Life, Arts & Entertainment, Stadiums &...",Phoenix,"1501 W Bell Rd\nPhoenix, AZ 85023","{'Monday': {'close': '18:00', 'open': '11:00'}...",33.638573,-112.092329,Turf Paradise Race Course,[],True,29,4.0,AZ,business
1,"{'Parking': {'garage': False, 'street': False,...",QbrM7wqtmoNncqjc6GtFaQ,"[Tires, Automotive, Fashion, Shopping, Departm...",Glendale,"18501 N 83rd Avenue\nGlendale, AZ 85308",{},33.648545,-112.234755,Sam's Club Members Only,[],True,3,3.5,AZ,business
2,"{'Parking': {'garage': False, 'street': False,...",7lbvsGKzhjuX3oJtaXJvOg,"[Women's Clothing, Men's Clothing, Fashion, Sh...",Tempe,"5000 S Arizona Mills Cir\nSte 590\nTempe, AZ 8...","{'Monday': {'close': '21:00', 'open': '10:00'}...",33.383123,-111.964485,Forever 21,[],True,7,3.5,AZ,business
3,{},gjxoKVsRJwEoa8zd9XxlAw,"[Pet Services, Pet Boarding/Pet Sitting, Pets]",Chandler,"912 W Sycamore Pl\nChandler, AZ 85225","{'Monday': {'close': '19:00', 'open': '06:00'}...",33.356472,-111.857818,Loving Hands Pet Care,[],True,4,5.0,AZ,business
4,{},V28yjMqyZnbCtabroJN_aA,"[Veterinarians, Pets]",Phoenix,"1745 W Glendale Ave\nPhoenix, AZ 85021",{},33.538493,-112.097232,Amec Mid-City Animal Hospital,[],True,3,5.0,AZ,business


In [73]:
business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15585 entries, 0 to 15584
Data columns (total 15 columns):
attributes       15585 non-null object
business_id      15585 non-null object
categories       15585 non-null object
city             15585 non-null object
full_address     15585 non-null object
hours            15585 non-null object
latitude         15585 non-null float64
longitude        15585 non-null float64
name             15585 non-null object
neighborhoods    15585 non-null object
open             15585 non-null bool
review_count     15585 non-null int64
stars            15585 non-null float64
state            15585 non-null object
type             15585 non-null object
dtypes: bool(1), float64(3), int64(1), object(10)
memory usage: 1.7+ MB


Not all columns are useful to us here. We are pretty sure we aren't interested in the specific address, attributes (facilities offered), the hours open, whether it's still open or not, the latitude and longitude, and the name of the restaurant. We also decide we aren't interested in drilling down to the postal code or neighborhood level either, so go ahead and drop those columns as well. You will now create a list of these columns and drop them from the DataFrame. Remember the axis and inplace arguments! Also, notice that there are fewer non-null categories than there are business_id. We'll come back to this point later.

In [74]:
# task: create a list of columns to drop, call it drop_cols
# then go ahead and drop those columns from the DataFrame
# two lines of code here:
drop_cols = ['attributes', 'city', 'full_address', 'hours', 'latitude', 'longitude', 'name', 'neighborhoods', 'open', 'type']
business.drop(drop_cols, axis=1, inplace=True)

In [75]:
business.head()

,business_id,categories,review_count,stars,state
0,O_X3PGhk3Y5JWVi866qlJg,"[Active Life, Arts & Entertainment, Stadiums &...",29,4.0,AZ
1,QbrM7wqtmoNncqjc6GtFaQ,"[Tires, Automotive, Fashion, Shopping, Departm...",3,3.5,AZ
2,7lbvsGKzhjuX3oJtaXJvOg,"[Women's Clothing, Men's Clothing, Fashion, Sh...",7,3.5,AZ
3,gjxoKVsRJwEoa8zd9XxlAw,"[Pet Services, Pet Boarding/Pet Sitting, Pets]",4,5.0,AZ
4,V28yjMqyZnbCtabroJN_aA,"[Veterinarians, Pets]",3,5.0,AZ


Okay, so now we have a much more manageable (width) of data. Remember our first task is to identify the most popular type of restaurant. We now have our eye on the categories column. We could simply count each individual category in this column, just as we did with our pet owners survey. But our categories column isn't quite in the same format as that data. Each item in categories is actually a string, as we demonstrate below, with each item we're interested in separated by commas.

In [76]:
business.head()

,business_id,categories,review_count,stars,state
0,O_X3PGhk3Y5JWVi866qlJg,"[Active Life, Arts & Entertainment, Stadiums &...",29,4.0,AZ
1,QbrM7wqtmoNncqjc6GtFaQ,"[Tires, Automotive, Fashion, Shopping, Departm...",3,3.5,AZ
2,7lbvsGKzhjuX3oJtaXJvOg,"[Women's Clothing, Men's Clothing, Fashion, Sh...",7,3.5,AZ
3,gjxoKVsRJwEoa8zd9XxlAw,"[Pet Services, Pet Boarding/Pet Sitting, Pets]",4,5.0,AZ
4,V28yjMqyZnbCtabroJN_aA,"[Veterinarians, Pets]",3,5.0,AZ


In [77]:
# print the first element of business['categories']
print(business.categories[0])

['Active Life', 'Arts & Entertainment', 'Stadiums & Arenas', 'Horse Racing']


In [78]:
# what is the type of this element?
type(business.categories[0])

list

In [79]:
# if the first element were a list, this would return 'Tours', not 'T'
business.categories[0][0]

'Active Life'

You're now going to use [this](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.split.html) pandas Series method to split these strings into lists.

In [ ]:
# task: split the categories column (a pandas Series) into lists of categories,
# e.g. the string "Category1, Category2" becomes the list ["Category1", "Category2"]
# hint: pass the argument pat = ', ?' in to split on a comma which may be followed by a space
business['categories'] = business['categories'].str.split(pat = ', ?')

In [ ]:
business['categories'].head()

See we now have a pandas Series of lists. Go ahead now and use the technique you learnt in the pet owners survey to print the top (most frequent) categories. We don't need to keep the raw, very long, list of categories, so go ahead and chain the method for counting the values and outputting the top twenty in one go. This may take your computer a minute or two.

In [ ]:
# task: count the frequency of each item in categories and keep the top twenty
# hint: the latter can be done by passing an argument to the head method
# assign the result to the variable category_counts - one line of code here:
category_counts = business['categories'].apply(pd.Series).stack().reset_index(drop=True).value_counts().head(20)

Now we introduce you to a new kind of plot: a bar plot. This gives us a nice visual representation of the frequency of the top 20 categories.

In [ ]:
category_counts.plot(kind='bar')

This has given us our first surprise. Our dataset contains more than just our desired industry (Restaurants). Someone hasn't thought to give us just the data we wanted. Over time this will surprise you less and less. In fact, as a data scientist, you'll become surprised when someone gives you a clean dataset containing only the data you wanted! Restaurants do seem to be our most frequent type of business, but represent fewer than 60,000 out of our original 188,000 businesses. We now figure we want to filter and keep just those businesses that have 'Restaurant' in their categories entry.

Before we start filtering on categories, however, remember we noticed early on that there were fewer non-null categories than there were business_id? How many null (missing) categories are there?

There are 541 business_id that have no category associated with them. Clearly these are not much use to us. First, we will have a quick look at the first few of them to check

In [ ]:
pd.isna(business['categories']).sum()

In [ ]:
business.loc[pd.isna(business['categories'])].head()

Sure enough there is no information in the categories column for these businesses. Go ahead and remove these 541 rows from the DataFrame. Assign the result back to the original DataFrame.

In [ ]:
# hint, the opposite of pd.isna() is ~pd.isna()
# one line of code here, assigning the result back to business
business = business.loc[business['categories'].apply(lambda x: 'Restaurants' in x), :]

Remember how you filtered our pet owner survey to select just dog owners? Now you're going to filter our business DataFrame to keep just those rows (businesses) that are restaurants. You'll do this in exactly the same way you did when testing whether 'dog' was in the list, although you'll keep all columns, not calculate a summary statistic, and will overwrite the original business DataFrame. Remember, we have a nice tidy list of categories now.

In [ ]:
business = business.loc[business['categories'].apply(lambda x: 'Restaurants' in x), :]

In [ ]:
business.shape

And now we can count the different categories for what should now just be restaurants

In [ ]:
restaurant_category_counts = business['categories'].apply(pd.Series).stack().reset_index(drop=True).value_counts().head(20)

In [ ]:
restaurant_category_counts.head()

In [ ]:
# 'Restaurants', of course, is our most frequent entry because, by definition, it is
# present in every case! Plot a bar chart now of the frequencies of the other values.
restaurant_category_counts[1:].plot(kind='bar')

We see we still have a few noise categories coming in, such as 'Food' and 'Nightlife'. But we can see that Pizza restaurants would seem to be a good bet for our marketing campaign trial! However, we also notice that Italian is pretty close behind, and using our domain knowledge, we figure that we'll use businesses that mention either Pizza or Italian.

<h2>Save desired business ID to file</h2>

Now it's time to create the final list of the business IDs we're interested in. These are (Restaurant) businesses that have either 'Italian' or 'Pizza' in their categories column.

In [ ]:
# task use a lambda function on the categories column again, but now you're not testing for
# 'Restaurant' but either 'Pizza or 'Italian'
# assign the result to italian_pizza (one line of code)
italian_pizza = business.loc[business['categories'].apply(lambda x: ('Pizza' in x) | ('Italian' in x)), :]

In [ ]:
italian_pizza.head()

In [ ]:
# Check the shape of the DataFrame
italian_pizza.shape

In [ ]:
# task: use panda's to_csv DataFrame method to save the result to a file
# use the index argument to suppress saving the index
# use the columns argument to save only the business_id, review_count, stars, and state columns
# one line of code here
italian_pizza.to_csv('business_list.csv', columns=['business_id', 'review_count', 'stars', 'state'], index=False)

<h1>Summary</h1>

Well done. You've done some pretty impressive data wrangling and some exploratory data analysis. You've read in some data and extracted a list of categories from one column, which is a pretty tricky thing to do! You've then explored the data and discovered a popular type of cuisine, then used that to extract just the relevant business IDs so you can use them as a key in your next step.